# Random Forest Classifier for Facial Emotion Recognition

This model is a Random Forest Classifier that has been trained to recognize face emotions. It classifies facial expressions into distinct emotion categories using a set of decision trees.

In this notebook, we also tried VGG to see if it makes any difference.

vgg_model.predict: Given a series of input photos, this method returns the predicted features from the VGG model.

In [26]:
import pandas as pd
import numpy as np


In [27]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from sklearn.ensemble import RandomForestClassifier


In [28]:
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))


In [29]:
data = pd.read_csv('fer2013.csv')


In [30]:
data['pixels'] = data['pixels'].apply(lambda pixel_string: np.array(pixel_string.split(' '), dtype=np.int32))

# Convert the single channel input images to three channels
data['pixels'] = data['pixels'].apply(lambda pixel_array: np.stack((pixel_array,)*3, axis=-1))


In [31]:
train_data = data[data['Usage'] == 'Training']
test_data = data[data['Usage'] == 'PublicTest']


In [32]:
x_train = np.array(train_data['pixels'].tolist()).reshape(-1, 48, 48, 3)
y_train = np.array(train_data['emotion'])
x_test = np.array(test_data['pixels'].tolist()).reshape(-1, 48, 48, 3)
y_test = np.array(test_data['emotion'])


In [33]:
x_train_preprocessed = preprocess_input(x_train)
x_test_preprocessed = preprocess_input(x_test)


In [34]:
x_train_features = vgg_model.predict(x_train_preprocessed)
x_test_features = vgg_model.predict(x_test_preprocessed)


113/113 [==============================] - 65s 578ms/step


In [35]:
n_samples = len(x_train_features)
n_features = np.prod(x_train_features.shape[1:])
x_train_features = x_train_features.reshape((n_samples, n_features))


rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(x_train_features, y_train)

RandomForestClassifier(random_state=42)

In [36]:
n_samples = len(x_test_features)
n_features = np.prod(x_test_features.shape[1:])
x_test_features = x_test_features.reshape((n_samples, n_features))


score = rf_model.score(x_test_features, y_test)
print('Accuracy: {:.2f}%'.format(score * 100))

Accuracy: 48.51%


In [41]:
y_pred = rf_model.predict(x_test_features)
y_train_pred = rf_model.predict(x_train_features)

In [42]:
from sklearn.metrics import classification_report

target_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
print('Training Classification')
print(classification_report(y_train, y_train_pred, target_names=target_names, labels=[0, 1, 2, 3, 4, 5, 6]))


Training Classification
              precision    recall  f1-score   support

       Angry       1.00      1.00      1.00      3995
     Disgust       1.00      1.00      1.00       436
        Fear       1.00      1.00      1.00      4097
       Happy       1.00      1.00      1.00      7215
         Sad       1.00      1.00      1.00      4830
    Surprise       1.00      1.00      1.00      3171
     Neutral       1.00      1.00      1.00      4965

    accuracy                           1.00     28709
   macro avg       1.00      1.00      1.00     28709
weighted avg       1.00      1.00      1.00     28709



In [38]:
from sklearn.metrics import classification_report

target_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
print('Test Classification')
print(classification_report(y_test, y_pred, target_names=target_names, labels=[0, 1, 2, 3, 4, 5, 6]))


Test Classification
              precision    recall  f1-score   support

       Angry       0.48      0.29      0.36       467
     Disgust       1.00      0.32      0.49        56
        Fear       0.46      0.32      0.38       496
       Happy       0.46      0.74      0.57       895
         Sad       0.44      0.40      0.42       653
    Surprise       0.66      0.71      0.69       415
     Neutral       0.44      0.35      0.39       607

    accuracy                           0.49      3589
   macro avg       0.56      0.45      0.47      3589
weighted avg       0.49      0.49      0.47      3589



### Analysis:

Precision: We can observe that the precision values range from 0.44 to 1.00 across different classes. This demonstrates the model's ability to categorize examples of each class accurately.

Recall : Recall values vary among classes and range from 0.29 to 0.74.

The F1-scores range from 0.36 to 0.69 across different classes.

The overall accuracy of the model on the test set is 0.49.

Overall, we can observe that the model does a slightly better job of categorizing the "Disgust" class, reaching perfect precision and a decent recall. However, other classes such as "Angry," "Fear," and "Neutral" have lower precision and recall ratings.


In [44]:
import joblib

joblib.dump(rf_model, "random_forest_model_withVGG.pkl")


['random_forest_model_withVGG.pkl']